In [ ]:
from langchain import Wikipedia
from langchain import SerpAPIWrapper
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.agents import (
    load_tools,
    Tool,
    initialize_agent,
    AgentType
)
from langchain.tools import BaseTool
from langchain.agents.react.base import DocstoreExplorer

import yfinance as yf
from pydantic import BaseModel, Field
from datetime import datetime, timedelta
from typing import Type

## Setting the LLM

In [ ]:
with open("openai_api.txt", "r") as f:
    OPENAI_API = f.read()

llm = OpenAI(
    model_name = "gpt-3.5-turbo-instruct",
    temperature = 0,
    openai_api_key = OPENAI_API
)

## Different Agent Types

Agents use an LLM to determine which actions to take and in what order. An action can either be using a tool and observing its output, or returning a response to the user. Here are the agents available in LangChain:

* _Zero-shot ReAct_: This agent uses the ReAct framework to determine which tool to use based solely on the tool's description. Any number of tools can be provided. This agent requires that a description is provided for each tool (it's the most general purpose action agent)

* _Structured input ReAct_: The structured tool chat agent is capable of using multi-input tools. Older agents are configured to specify an action input as a single string, but this agent can use a tools' argument schema to create a structured action input. This is useful for more complex tool usage, like precisely navigating around a browser.

* _OpenAI Functions_: Certain OpenAI models (like gpt-3.5-turbo-0613 and gpt-4-0613) have been explicitly fine-tuned to detect when a function should be called and respond with the inputs that should be passed to the function. The OpenAI Functions Agent is designed to work with these models.

* _Conversational_: This agent is designed to be used in conversational settings. The prompt is designed to make the agent helpful and conversational. It uses the ReAct framework to decide which tool to use, and uses memory to remember the previous conversation interactions.

* _Self-ask with Search_: This agent utilizes a single tool that should be named Intermediate Answer. This tool should be able to lookup factual answers to questions. This agent is equivalent to the original self-ask with search paper, where a Google search API was provided as the tool.

* _ReAct Document Store_: This agent uses the ReAct framework to interact with a docstore. Two tools must be provided: a Search tool and a Lookup tool (they must be named exactly as so). The Search tool should search for a document, while the Lookup tool should lookup a term in the most recently found document. This agent is equivalent to the original ReAct paper, specifically the Wikipedia example.

In [ ]:
## Creating the Tools

search = SerpAPIWrapper(serpapi_api_key=open("serpapi_api.txt", 'r').read())
search_tool = Tool(
    name = "Current Search",
    func = search.run,
    description = "useful for when you need to answer questions about current events or the current state of the world"
)

tools = load_tools(
    tool_names = ["llm-math"],
    llm = llm
)

tools.append(search_tool)

### `zero-shot-react-description`

This agent uses the ReAct framework to determine which tool to use based solely on the tool's description. Any number of tools can be provided. This agent requires that a description is provided for each tool.

As described earlier, we use this agent to perform “zero-shot” tasks on some input. That means the agent considers one single interaction with the agent — it will have `no memory`.

In [ ]:
zero_shot_agent = initialize_agent(
    agent = "zero-shot-react-description",
    tools = tools,
    llm = llm,
    verbose = True,
    max_iterations = 3
)

In [ ]:
## Gets Error

result = zero_shot_agent(
    "What is the multiplication of the ratio of the prices of stocks 'Tesla' "
    "and 'Amazon' and the ratio of the age of Brad Bitt and Leonardo Di Caprio?"
)

In [ ]:
result = zero_shot_agent("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")

### `conversational-react-description`

This agent is designed to be used in `conversational` settings. The prompt is designed to make the agent helpful and conversational. It uses the ReAct framework to decide which tool to use, and uses memory to remember the previous conversation interactions.

The zero-shot agent works well but lacks `conversational memory`. This lack of memory can be problematic for chatbot-type use cases that need to remember previous interactions in a conversation.

By using the `conversational-react-description` we simulate memory.

In [ ]:
## Initializing Memory Buffer | `memory_key` default is `history` (the variable to store the conversation)

memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
conversational_agent = initialize_agent(
    agent = "conversational-react-description",
    tools = tools,
    llm = llm,
    verbose = True,
    max_iterations = 3,
    memory = memory,
)

In [ ]:
result = conversational_agent("Please provide me the stock prices of Tesla?")

In [ ]:
result = conversational_agent("Of Apple?")

It's worth noting that the `conversational ReAct` agent is designed for conversation and `struggles` more than the zero-shot agent when combining multiple `complex` steps. We can see this if we ask the agent to answer our earlier question:

In [ ]:
# Get's an Error

result = conversational_agent(
    "What is the multiplication of the ratio of the prices of stocks 'Tesla' "
    "and 'Amazon' and the ratio of the age of Brad Bitt and Leonardo Di Caprio?"
)

we can see that is trying to use the calculator first rather the SQL Chain, so it's producing an error.

### `react-docstore`

This agent uses the ReAct framework to interact with a docstore. Two tools must be provided: a `Search` tool and a `Lookup` tool (they must be named exactly as so). The `Search` tool should search for a document, while the `Lookup` tool should lookup a term in the most recently found document.

`LangChain docstores` allow us to store and retrieve information using traditional retrieval methods. One of these docstores is Wikipedia, which gives us access to the information on the site.

In [ ]:
## Initializing the Docstore Chain

docstore=DocstoreExplorer(Wikipedia())

In [ ]:
## Creating the Tools for `Search` and `Lookup`

tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search"
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup"
    )
]

In [ ]:
docstore_agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent = "react-docstore",
    verbose = True
)

In [ ]:
docstore_agent("Author David Chanoff has collaborated with a U.S. Navy admiral who served as the ambassador to the United Kingdom under which President?")

In [ ]:
docstore_agent("What were Archimedes' last words?")

### `self-ask-with-search`

This agent utilizes a single tool that should be named `Intermediate Answer`. This tool should be able to lookup factual answers to questions.

In [ ]:
## Initializing the Search Chain

search = SerpAPIWrapper(serpapi_api_key = open("serpapi_api.txt").read())

In [ ]:
## Creating the Search Tool
tools = [
    Tool(
        name = "Intermediate Answer",
        description = "useful for when you need to ask with search",
        func = search.run
    )
]

In [ ]:
self_ask_with_search = initialize_agent(
    tools = tools,
    llm = llm,
    agent = "self-ask-with-search",
    verbose = True
)

In [ ]:
self_ask_with_search("who lived longer; Plato, Socrates, or Aristotle?")

We can see the multi-step process of the agent. It performs multiple follow-up questions to hone in on the final answer.

### `openai-functions`

Certain OpenAI models (like gpt-3.5-turbo-0613 and gpt-4-0613) have been fine-tuned to detect when a function should be called and respond with the inputs that should be passed to the function. In an API call, you can describe functions and have the model intelligently choose to output a JSON object containing arguments to call those functions. The goal of the OpenAI Function APIs is to more reliably return valid and useful function calls than a generic text completion or chat API.

In [ ]:
## Setting the Tools

search = SerpAPIWrapper(serpapi_api_key=open("serpapi_api.txt", 'r').read())

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    )
]

In [ ]:
## Setting the `functions` LLM

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613", openai_api_key=open("openai_api.txt", 'r').read())

In [ ]:
## Setting the Agent
agent_executor = initialize_agent(
    tools = tools,
    llm = llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

agent_executor.invoke({"input": "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"})

In [ ]:
## A More Complex Example: Defining Custom Functions


def get_current_stock_price(ticker):
    """Method to get current stock price"""

    ticker_data = yf.Ticker(ticker)
    recent = ticker_data.history(period="1d")
    return {"price": recent.iloc[0]["Close"], "currency": ticker_data.info["currency"]}


def get_stock_performance(ticker, days):
    """Method to get stock price change in percentage"""

    past_date = datetime.today() - timedelta(days=days)
    ticker_data = yf.Ticker(ticker)
    history = ticker_data.history(start=past_date)
    old_price = history.iloc[0]["Close"]
    current_price = history.iloc[-1]["Close"]
    return {"percent_change": ((current_price - old_price) / old_price) * 100}


# Creating Custom Tools

class CurrentStockPriceInput(BaseModel):
    """Inputs for get_current_stock_price"""

    ticker: str = Field(description="Ticker symbol of the stock")


class CurrentStockPriceTool(BaseTool):
    name = "get_current_stock_price"
    description = """
        Useful when you want to get current stock price.
        You should enter the stock ticker symbol recognized by the yahoo finance
        """
    args_schema: Type[BaseModel] = CurrentStockPriceInput

    def _run(self, ticker: str):
        price_response = get_current_stock_price(ticker)
        return price_response

    def _arun(self, ticker: str):
        raise NotImplementedError("get_current_stock_price does not support async")


class StockPercentChangeInput(BaseModel):
    """Inputs for get_stock_performance"""

    ticker: str = Field(description="Ticker symbol of the stock")
    days: int = Field(description="Timedelta days to get past date from current date")


class StockPerformanceTool(BaseTool):
    name = "get_stock_performance"
    description = """
        Useful when you want to check performance of the stock.
        You should enter the stock ticker symbol recognized by the yahoo finance.
        You should enter days as number of days from today from which performance needs to be check.
        output will be the change in the stock price represented as a percentage.
        """
    args_schema: Type[BaseModel] = StockPercentChangeInput

    def _run(self, ticker: str, days: int):
        response = get_stock_performance(ticker, days)
        return response

    def _arun(self, ticker: str):
        raise NotImplementedError("get_stock_performance does not support async")


# Creating the Agent
llm = ChatOpenAI(model="gpt-3.5-turbo-0613", temperature=0, openai_api_key=open("openai_api.txt", 'r').read())
tools = [CurrentStockPriceTool(), StockPerformanceTool()]
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [ ]:
agent.run("What is the current price of Microsoft stock? How it has performed over past 6 months?")

In [ ]:
agent.run("Give me recent stock prices of Google and Meta?")